In [1]:
import pandas as pd
import numpy as np
import re
import datetime

In [2]:
def create_submission_df(crash_data_df, date_start='2019-07-01', date_end='2020-01-01'):
    '''Takes crash data and creates a data frame in the format needed for submission'''
    
    #calculate centroid for ambulance placement
    lat_centroid = list(crash_data_df.latitude.quantile(q=[1/7,2/7,3/7,4/7,5/7,6/7]))
    lon_centroid = list(crash_data_df.longitude.quantile(q=[1/7,2/7,3/7,4/7,5/7,6/7]))
    
    # Create Date range covering submission period set
    dates = pd.date_range(date_start, date_end, freq='3h')
        
    # Create submission dataframe
    submission_df = pd.DataFrame({
        'date':dates
    })
    for ambulance in range(6):
        # Place an ambulance in the center of the city:
        submission_df['A'+str(ambulance)+'_Latitude'] = lat_centroid[ambulance]
        submission_df['A'+str(ambulance)+'_Longitude'] = lon_centroid[ambulance]
    return submission_df

In [3]:
def create_submission_csv(submission_df, model_name='baseline', path='../Outputs/'):
    '''Takes dataframe in submission format and outputs a csv file with matching name'''
    current_time = datetime.datetime.now()
    filename = f'{current_time.year}{current_time.month}{current_time.day}_{model_name}.csv'
    submission_df.to_csv(path+filename,index=False)

In [4]:
# Load raw crash data from csv
crash_data_df = pd.read_csv('../Inputs/Train.csv', parse_dates=['datetime'])

In [5]:
lon_centroid = crash_data_df.longitude.quantile(q=[1/7,2/7,3/7,4/7,5/7,6/7])
lon_centroid

0.142857    36.763869
0.285714    36.808880
0.428571    36.833825
0.571429    36.859843
0.714286    36.888859
0.857143    36.931377
Name: longitude, dtype: float64

In [10]:
# call function to create submission df and csv output
submission_df = create_submission_df(crash_data_df)
create_submission_csv(submission_df,'quantiles')

In [7]:
# View contents of submission
submission_df.head()

,date,A0_Latitude,A0_Longitude,A1_Latitude,A1_Longitude,A2_Latitude,A2_Longitude,A3_Latitude,A3_Longitude,A4_Latitude,A4_Longitude,A5_Latitude,A5_Longitude
0,2019-07-01 00:00:00,-1.329937,36.763869,-1.305619,36.80888,-1.285169,36.833825,-1.263771,36.859843,-1.245356,36.888859,-1.203744,36.931377
1,2019-07-01 03:00:00,-1.329937,36.763869,-1.305619,36.80888,-1.285169,36.833825,-1.263771,36.859843,-1.245356,36.888859,-1.203744,36.931377
2,2019-07-01 06:00:00,-1.329937,36.763869,-1.305619,36.80888,-1.285169,36.833825,-1.263771,36.859843,-1.245356,36.888859,-1.203744,36.931377
3,2019-07-01 09:00:00,-1.329937,36.763869,-1.305619,36.80888,-1.285169,36.833825,-1.263771,36.859843,-1.245356,36.888859,-1.203744,36.931377
4,2019-07-01 12:00:00,-1.329937,36.763869,-1.305619,36.80888,-1.285169,36.833825,-1.263771,36.859843,-1.245356,36.888859,-1.203744,36.931377


In [9]:
# functions for saving post submission details to a dataframe
def get_submission_details(copypaste):
    split_input = re.split("\s", copypaste)
    id_str = split_input[0]
    score_float = split_input[-1]
    file_name = filename = re.findall('\w*.csv\w*', copypaste)[-1]
    comment_str = ''
    submitted_datetime = ''
    submission_row = [id_str, file_name, score_float, submitted_datetime, comment_str]
    return submission_row
def submission_dataframe(rows,columns=['ID', 'FILE', 'SCORE', 'SUBMITTED', 'COMMENT']):
    return pd.DataFrame(data=[rows], columns = columns)

def create_dataframe(copypaste):
    return submission_dataframe(get_submission_details(copypaste))

# the below needs work
def add_to_submission_dataframe(df, rows, columns=['ID', 'FILE', 'SCORE', 'SUBMITTED', 'COMMENT']):
    return df.append(zip([columns,rows]), ignore_index=True)

def update_dataframe(df, copypaste):
    rows = get_submission_details(copypaste)
    return add_to_submission_dataframe(df, rows)